<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Data Preparation and Discovery</b>
<br>
<b style = 'font-size:20px;font-family:Arial;color:#E37C4D'>Using Teradataml python package</b>
</header>


<b style = 'font-size:18px;font-family:Arial;color:#E37C4D'>Introduction</b>
<p style = 'font-size:16px;font-family:Arial'>This is a demonstration of the teradataml package that is designed for data management, exploration, and execution of analytic functions.</p>

<p style = 'font-size:16px;font-family:Arial'>The current version of the teradataml package includes <b>over 100 functions</b>, organized into these functional areas:</p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Utility and database management functions</li>
    <li>Data exploration and preparation functions</li>
    <li>Analytic functions across Vantage</li>
</ul>

<p style = 'font-size:16px;font-family:Arial'>These functions leverage the full power and scale inside Vantage without:</p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Costly, slow export of data out of the DBMS</li>
    <li>Being limited by client platform resources</li>
    <li>Having to write complex SQL</li>
</ul>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Contents</b></p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>Configuring the Environment</li>
    <li>Initiate a connection to Vantage</li>
    <li>Create and Load Tables</li>
    <li>Data Discovery</li>
    <li>Working with Data</li>
    <li>Advanced Data Preparation</li>
    <li>Visualizing results.</li>
</ol>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>1. Configuring the Environment</b>
<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
import json
import getpass
import os
import warnings
#Suppress Warnings
warnings.filterwarnings('ignore')

import pandas as pd

from teradataml.dataframe.dataframe import DataFrame
from teradataml.dataframe.dataframe import in_schema
from teradataml.context.context import create_context, remove_context
from teradataml.dataframe.copy_to import copy_to_sql
from teradataml.dataframe.fastload import fastload

from sqlalchemy import func

import seaborn as sns
%matplotlib inline

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>2. Initiate a connection to Vantage</b>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, then use down arrow to go to next cell.</p>

In [ ]:
%run -i ../startup.ipynb

<p style = 'font-size:16px;font-family:Arial'>Below command will create a context to the Vantage connection.</p>

In [ ]:
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

In [ ]:
%sql SET query_band='DEMO=Consumption_Forecasting_BYOM.ipynb;' UPDATE FOR SESSION;

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>3. Create and Load Tables</b>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>3.1  Create Demo Transaction data - simulated funds transfers.  Use FastLoad to create and import data</b></p>

<p style = 'font-size:16px;font-family:Arial'>Fastload protocol is nice for row counts over 100K - shown here as an illustration. These Teradata functions have lots of parameters to help control behavior - the if_exists parameter is nice so we don't have to explictly drop the table before loading it - or we can append, etc. We can also use copy_to_sql for smaller row counts and more flexibility</p>

In [ ]:
# Read the CSV data into a local pandas dataframe
ip_data = pd.read_csv('data/Transactions_60K.csv')

# Little bit of code that creates an index
ip_data['txn_id'] = range(1, len(ip_data) + 1)

fastload(ip_data,
         table_name='ip_data', 
         primary_index='txn_id',
         if_exists = 'replace',
         open_sessions=2)

In [ ]:
pd.read_sql("SELECT COUNT(*) FROM ip_data;", eng)

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>3.2  Create Simulated Customer Data - load from tdf using SQL to create the table</b></p>
<p style = 'font-size:16px;font-family:Arial'>In the example above, we created our table automatically by calling the Fastload.  These functions allow us to define data types, encoding, and other parameters.  However if we want more control, we can use SQL to create the table.  In the below example, we need the "ST_GEOMETRY" data type which isn't supported by python. Hence we use Terdata SQL to overcome this limitation of python.</p>

In [ ]:
qry = '''
CREATE MULTISET TABLE CUSTOMER, NO FALLBACK,
     NO BEFORE JOURNAL,
     NO AFTER JOURNAL,
     CHECKSUM = DEFAULT,
     DEFAULT MERGEBLOCKRATIO
(
    CUSTOMER_ID DECIMAL(18,0) NOT NULL,
    F_NAME VARCHAR(30),
    L_NAME VARCHAR(30),
    VALIDITY VARCHAR(60),
    CUST_ZIP VARCHAR(5),
    CUST_LOCATION ST_GEOMETRY,
    ETHNICITY VARCHAR(20),
    GENDER CHAR(1),
    CHURN_FLAG VARCHAR(1)
)
PRIMARY INDEX(CUSTOMER_ID);
'''

try:
    eng.execute(qry)
except:
    eng.execute('DROP TABLE CUSTOMER;')
    eng.execute(qry)

<p style = 'font-size:16px;font-family:Arial'>Load the customer data - we're using the table created above, and reading the tdf file directly. Note that we have flexibility on different delimiters - in this case it is a tab.</p>

In [ ]:
copy_to_sql(pd.read_csv('data/CUSTOMER.tdf', sep='\t'), table_name='CUSTOMER')

In [ ]:
pd.read_sql("SELECT COUNT(*) FROM CUSTOMER;", eng)

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>3.3  Create Simulated Customer Comment Table.  Use copy_to_sql to create the table and load the data</b></p>

In [ ]:
copy_to_sql(pd.read_csv('data/CUST_COMMENT.csv'),
            table_name='CUST_COMMENT',
            if_exists = 'replace')

In [ ]:
pd.read_sql("SELECT COUNT(*) FROM CUST_COMMENT;", eng)

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>3.4  Additional Simulated Data - Server Locations.  Use SQL to handle the ST_GEOMETRY data type </b></p>
<p style = 'font-size:16px;font-family:Arial'>This shows an example where ST_GEOMETRY data type is not supported by python, so we can use Teradata SQL to overcome the limitations</p>

In [ ]:
qry = '''
CREATE MULTISET TABLE SERVER, NO FALLBACK,
     NO BEFORE JOURNAL,
     NO AFTER JOURNAL,
     CHECKSUM = DEFAULT,
     DEFAULT MERGEBLOCKRATIO
(
    SERVER_ID VARCHAR(5) NOT NULL,
    SERVER_ZIP VARCHAR(5),
    SERVER_LOCATION ST_GEOMETRY
)
PRIMARY INDEX(SERVER_ID);
'''

try:
    eng.execute(qry)
except:
    eng.execute('DROP TABLE SERVER;')
    eng.execute(qry)

In [ ]:
#load the data - read the csv file using pandas read_csv
srvr = pd.read_csv('data/SERVER.csv')
srvr.rename(columns = {'SERVER ZIP':'SERVER_ZIP','SERVER_LAT':'SERVER_LOCATION'}, inplace = True)

copy_to_sql(srvr, table_name='SERVER')

In [ ]:
pd.read_sql("SELECT COUNT(*) FROM SERVER;", eng)

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>4. Data Discovery</b>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Look at table statistics, sample data, simple lookups</b></p>

<p style = 'font-size:16px;font-family:Arial'>One of the most powerful features of the teradataml functions is that they push processing down to the Teradata system, allowing users to perform analysis without pulling all the data back to the client.
<br>
The following cell creates a pointer(virtual dataframe) to the <b>ip_data</b> table server. We retrieve the size of the table and retrieve a small sample i.e. 5 rows back to this python environment in order to have a look at the sample data.</p>

In [ ]:
# Get a teradata DataFrame - this creates a local reference to the large table on the server.
tdf_ip_data = DataFrame("ip_data")

# Check the data - size and sample rows without returning all the data
print(tdf_ip_data.shape)

# Return a small set of the data back to a traditional Pandas DF for full-featured formatting.
tdf_ip_data.to_pandas(num_rows = 5).head()

<p style = 'font-size:16px;font-family:Arial'>Please scroll down to the end of the notebook for detailed column descriptions of the above dataset.</p>

In [ ]:
# Check for null values.
tdf_ip_data.info(null_counts = True)

<p style = 'font-size:16px;font-family:Arial'>The above result shows that there are no nulls in the dataset.
    <br>
    <br>
Generate Stats using Teradata DataFrame describe method - similar to pandas but it runs on the server, we don't need to retrieve all the data. This cell shows the column-wise statistics.</p>

In [ ]:
tdf_ip_data.describe()

<p style = 'font-size:16px;font-family:Arial'>Apply a set of expressions on the virtual dataframe using loc (pandas set processing technique) to grab all fraudulent values. The logic here will filter fraudulent transactions of type 'TRANSFER' - as a view on the server, not actually move the data at all. Calling head(2) will only retrieve two values from the sever.
<br>
<br>
The following cell shows a sample of 2 Fraud transactions which were of type TRANSFER.</p>

In [ ]:
tdf_ip_data.loc[(tdf_ip_data.isFraud == 1) & (tdf_ip_data.type == 'TRANSFER')].head(2)

<p style = 'font-size:16px;font-family:Arial'>The following cell shows a sample of 2 Fraud transactions with only the 3 columns(amount, isFraud, type).

In [ ]:
#filter the dataframe, then only retrive two rows of results
tdf_ip_data.loc[tdf_ip_data.isFraud == 1].filter(items = ['amount', 'isFraud', 'type']).head(2)

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>5. Working with Data at Scale</b>

<p style = 'font-size:16px;font-family:Arial'>Act on our data sets without having to return all of the data, and leverage the computing power of the Teradata Vantage cluster.</p>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>5.1 Aggregations</b></p>

<p style = 'font-size:16px;font-family:Arial'>We can use these "fluent" methods to keep the code as brief and expressive as possible. The following cell counts each instance of fraud grouped by transaction type. Note the only data that actually moves out of the database is the final count() aggregation.</p>

In [ ]:
tdf_ip_data.loc[tdf_ip_data.isFraud == 1].filter(items = ['amount', 'isFraud', 'type']).groupby('type').count()

<p style = 'font-size:16px;font-family:Arial'> The following cell gives us min and max transaction amounts by transaction type. We can use multiple aggregates in the agg() function call.</p>

In [ ]:
tdf_ip_data.loc[tdf_ip_data.isFraud == 1].filter(items = ['amount', 'isFraud', 'type']).groupby('type').agg({'amount' : ['min', 'max']})

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>5.2 Simple Transformations</b></p>

<p style = 'font-size:16px;font-family:Arial'>Create new "Virtual Dataframes" that are the result of dropping columns or adding new ones via simple expression. The following cell creates a new Virtual DataFrame by dropping few columns.</p>

In [ ]:
clean_data = tdf_ip_data.loc[tdf_ip_data.isFraud == 1].drop(['nameDest', 'nameOrig', 'isFlaggedFraud'], axis = 1)
clean_data.head(2)

<p style = 'font-size:16px;font-family:Arial'>The following cell assigns a new column which is the difference between newbalanceDest and amount of transaction.</p>

In [ ]:
clean_data = clean_data.assign(diff = clean_data['newbalanceDest'] - clean_data['amount'])
clean_data.head(2)

<p style = 'font-size:16px;font-family:Arial'>The following cell creates a new column which is the binary representation of transaction type.</p>

In [ ]:
clean_data = clean_data.assign(btype = clean_data['type'].str.contains('CASH_OUT'))
clean_data.head(5)

<p style = 'font-size:16px;font-family:Arial'>Here, binary representation of 1 means CASH_OUT and 0 means TRANSFER. This is imilar to Ordinal Encoding.</p>

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>5.3 Joins</b></p>
<p style = 'font-size:16px;font-family:Arial'>Join dataframes using python pandas-style join methods. These are teradataml DataFrame methods and run completely in database.</p>

In [ ]:
tdf_customer = DataFrame('CUSTOMER')
tdf_customer.head(5)

In [ ]:
tdf_cust_comment = DataFrame('CUST_COMMENT')
tdf_cust_comment.head(5)

<p style = 'font-size:16px;font-family:Arial'>The following cell performs a join between CUSTOMER and CUST_COMMENT tables on CUSTOMER_ID index.</p>

In [ ]:
#Do an inner join and drop up fields we don't need.
tdf_comment_full = tdf_cust_comment.join(other = tdf_customer, 
                                         on = ['CUSTOMER_ID = CUSTOMER_ID'], 
                                         how = 'inner', 
                                         lsuffix = 'cID_', 
                                         rsuffix = 'cOM_')
tdf_comment_full.drop(['COMMENT_ID', 'cOM__CUSTOMER_ID', 'CHANNEL_ID', 'GENDER', 'CHANNEL_TYPE', 'ETHNICITY', 'VALIDITY'], axis = 1)

<p style = 'font-size:16px;font-family:Arial'>The above output shows the join of two tables CUSTOMER and CUST_COMMENT. Note that tdf_customer and tdf_cust_comment are just pointers to the Teradata Dataframe. The data is not moved in this process.

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>6. Advanced Data Preparation</b>

<p style = 'font-size:16px;font-family:Arial'>The TeradataML Python package has exposed many powerful SQL data transformation functions to the user.  These functions can be applied to Teradata Dataframes to operate on data at scale in the database.
See the documentation for a full list of functions (including aggregate, arithmetic, Bit/Byte, Date and Time, Hash, Regular Expression, and String Functions).</p>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>6.1 Aggregate Functions:</b></p>

In [ ]:
#This import statement is also above, but rewritten here for emphasis. These functions are applied to the
#Teradata dataframe via the SQLAlchemy func class

from sqlalchemy import func

#reuse our datasets from above
clean_data.head()

<p style = 'font-size:16px;font-family:Arial'>The Pearson correlation coefficient (r) is the most common way of measuring a linear correlation. It is a number between -1 and 1 that measures the strength and direction of the relationship between two variables. Pearson Correlation Coefficient - in this example, for our fraudulent data set: What's the correlation between the original balance and the transfer amount?</p>

In [ ]:
corr_func = func.corr(clean_data['oldbalanceOrg'].expression, clean_data['amount'].expression)


#Setting drop_columns = True here
df_corr = clean_data.assign(drop_columns = True, corr_ = corr_func)

print(df_corr)

<p style = 'font-size:16px;font-family:Arial'>Here, the Pearson Correlation Coefficient is higher and hence it means when original balance is high, transfer amount is also high. Both the variables are positively correlated.
<br>
<br>
Kurtosis: Let's see what the variance from the normal distribution looks like for our full data set for example - transfer amounts, grouped by the transfer type. Normal distribution has a kurtosis of 0, negative indicates less outliers, positive represents larger outliers.</p>

In [ ]:
kurtosis_func = func.kurtosis(tdf_ip_data['amount'].expression)

#Can also set drop_columns positionally
df_kurtosis = tdf_ip_data.groupby('type').assign(True, kurtosis_xfer_amt = kurtosis_func)

print(df_kurtosis)

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>6.2 Arithmetic Functions</b></p>

<p style = 'font-size:16px;font-family:Arial'>Natural Log: Let's add an additional column which is the natural log of the transfer amount. Since we see from above in calling the describe() method for amount column, min is .79, max is 36946551.76. We can use the natural log to create a tighter range of values for possible use in analysis.</p>

In [ ]:
tdf_new = tdf_ip_data.assign(ln_amount = func.ln(tdf_ip_data['amount'].expression))

print(tdf_new.filter(items = ['amount', 'ln_amount']))

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>6.3 String Functions</b></p>

<p style = 'font-size:16px;font-family:Arial'>Converting payment type to lower case.</p>

In [ ]:
tdf_lower = tdf_ip_data.assign(False, type_lower = func.lower(tdf_ip_data['type'].expression))
print(tdf_lower.filter(items = ['type', 'type_lower']))

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>6.4 Regular Expression Functions</b></p>

<p style = 'font-size:16px;font-family:Arial'>Return the substring based on a regular expression. In our demo data, the "nameDest" has a character code as the first character of the account name (example M1057061069)</p>

In [ ]:
regexp_func = func.regexp_substr(tdf_ip_data['nameDest'].expression, '^[A-Z]{1}')

tdf_regex = tdf_ip_data.assign(False, acct_ind = regexp_func)

print(tdf_regex.filter(items = ['nameDest', 'acct_ind']))

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>7. Visualizations</b>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.1 - Example - Geospatial query to return plottable data</b></p>

<p style = 'font-size:16px;font-family:Arial'>The following code calculates the Spherical Distance between the customers and servers.</p>

In [ ]:
qry = '''
SELECT C.CUSTOMER_ID, 
  S.SERVER_ID, 
  CAST(C.CUST_LOCATION.ST_SphericalDistance(S.SERVER_LOCATION)/1000 AS DECIMAL(10,0))AS KM_DISTANCE 
FROM CUSTOMER C, SERVER S
WHERE S.SERVER_ZIP = C.CUST_ZIP
'''

tdf_distance = DataFrame.from_query(qry)

In [ ]:
tdf_distance.head(5)

In [ ]:
#Sort by greatest distance away
tdf_distance.sort('KM_DISTANCE', ascending = False).head(5)

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.2 Use Pandas/seaborn to create visualizations inline</b></p>

<p style = 'font-size:16px;font-family:Arial'> The following code creates a graph that shows the distribution of distance of customer and server. A majority of customers and servers are in a 4000 km range.</p>

In [ ]:
sns.distplot(tdf_distance.to_pandas()['KM_DISTANCE'].astype(float), bins=50);

In [ ]:
#Do a bunch of work to filter, group, aggregate, retrieve, and format our chart
tdf_ip_data.drop(['step', 'isFraud', 'isFlaggedFraud'], axis = 1).groupby('type').sum().to_pandas().set_index('type').plot(kind = 'bar');

<p style = 'font-size:16px;font-family:Arial'>The above graph shows the sum of amount, oldbalanceOrig, newbalanceOrig, oldbalanceDest, and newbalanceDest grouped by transcation type.</p>

<hr>
<b style = 'font-size:18px;font-family:Arial;color:#E37C4D'>Cleanup</b>
<p style = 'font-size:16px;font-family:Arial'>It is a good practice to drop tha tables that we created and to remove the context that we created to connect to Vantage.</p>

In [ ]:
eng.execute('DROP TABLE CUSTOMER;')

In [ ]:
eng.execute('DROP TABLE CUST_COMMENT;')

In [ ]:
eng.execute('DROP TABLE SERVER;')

In [ ]:
eng.execute('DROP TABLE ip_data;')

In [ ]:
remove_context()

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Dataset:</b>

- `txn_id`: transaction id
- `step`: maps a unit of time in the real world. In this case 1 step is 1 hour of time. Total steps 744 (31 days simulation).
- `type`: CASH-IN, CASH-OUT, DEBIT, PAYMENT and TRANSFER
- `amount`: amount of the transaction in local currency
- `nameOrig`: customer who started the transaction
- `oldbalanceOrig`: customer's balance before the transaction
- `newbalanceOrig`: customer's balance after the transaction
- `nameDest`: customer who is the recipient of the transaction
- `oldbalanceDest`: recipient's balance before the transaction
- `newbalanceDest`: recipient's balance after the transaction
- `isFraud`: identifies a fraudulent transaction (1) and non fraudulent (0)
- `isFlaggedFraud`: flags illegal attempts to transfer more than 200,000 in a single transaction

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Teradata-Package-for-Python-User-Guide-17.20'>https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Teradata-Package-for-Python-User-Guide-17.20</a></li>
</ul>

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">Copyright © Teradata Corporation - 2023. All Rights Reserved.</footer>